In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from pandas import DataFrame
from pandas import json_normalize
# from pandas.io.json import json_normalize
print ("pymongo version:", pymongo.version)

pd.set_option('display.max_columns', 1010)
pd.set_option('display.max_rows', 1010)

/mnt/c/projects/riskmodel/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


pymongo version: 3.10.1


### Check status mongodb
* sudo service mongodb status
* sudo service mongodb start

In [ ]:
mongo_client = MongoClient('localhost', 27017)
host_info = mongo_client['HOST']
print ("\nhost:", host_info)

### List databases

In [ ]:
print(mongo_client.list_database_names())

In [ ]:
legalthings_db = mongo_client["legalthings"]

### Show collections in database

In [ ]:
legalthings_db.list_collection_names ()

### Make dataframe of collection

In [ ]:
df = DataFrame(list(legalthings_db.incorporation_processes.find({})))

In [ ]:
df.shape

In [ ]:
df.head(n=2)

#### Normalize JSON

However the subdocuments will just appear as JSON inside the subdocument cell. If you want to flatten objects so that subdocument properties are shown as individual cells you can use json_normalize without any parameters.

In [ ]:
datapoints = list(legalthings_db.incorporation_processes.find({}))

In [ ]:
df = json_normalize(datapoints)

### First exploration of the data
* Important note: collection is still small subset of the total collection, otherwise we run into memory problems
* In a later stadium, most important data is extracted the total collection

In [ ]:
df.shape, len(df['_id'].unique()), len(df['name'].unique()), df['title'].unique(), df['current.title'].unique()

Drop columns with all missings

In [ ]:
df = df.dropna(axis=1, how='all')

In [ ]:
df.shape

Get duplicate company names

In [ ]:
df[df['name'].duplicated(keep=False)].sort_values('name').head()

Get companies with status 'Cliënt rond de betaling af' --> probably shows whole funnel

In [ ]:
companies_funnel_all = (
    df[df['current.title'].isin(['Cliënt rond de betaling af', 'Cliënt rondt de betaling af'])]
    ['name']
    .unique()
)

len(companies_funnel_all)

In [ ]:
df[df['name'] == companies_funnel_all[0]].dropna(axis=1, how='all')

#### Still some nested data --> show 1 collection to explore JSON structure

In [ ]:
legalthings_db.incorporation_processes.find_one()

Example unnest 'scenario.alias' and add some extra information
* https://stackoverflow.com/questions/47242845/pandas-io-json-json-normalize-with-very-nested-json

In [ ]:
json_normalize(datapoints, [['scenario', 'alias']], ['_id', 'title','name']).head()

#### Unnest company with probably a lot of information within nested fields

In [ ]:
df[df['name'] == companies_funnel_all[0]].dropna(axis=1, how='all')

In [ ]:
filter_query = { "name": "MEA" }

datapoint = list(legalthings_db.incorporation_processes.find(filter_query))

#### private_data.meta_data.kvk.type_toelichting

In [ ]:
df[df['name'] == companies_funnel_all[0]].dropna(axis=1, how='all')['private_data.meta_data.kvk.type_toelichting']

#### private_data.meta_data.ah --> Nice, NAW information

In [ ]:
json_normalize(datapoint, [['private_data', 'meta_data', 'ah']], ['_id', 'title','name'])

#### private_data.assets.invoice.versions

In [ ]:
json_normalize(datapoint, [['private_data', 'assets', 'invoice', 'versions']], ['_id', 'title','name'])

#### private_data.assets.invoice.created.user.employment

In [ ]:
json_normalize(datapoint, [['private_data', 'assets', 'invoice', 'created', 'user', 'employment']])

#### private_data.assets.invoice.last_updated.user.employment

In [ ]:
json_normalize(datapoint, [['private_data', 'assets', 'invoice', 'last_updated', 'user', 'employment']])

#### private_data.products

In [ ]:
json_normalize(datapoint, [['private_data', 'products']])

#### private_data.payment.products

In [ ]:
json_normalize(datapoint, [['private_data', 'payment', 'products']])

#### private_data.payment.user.authz_groups

In [ ]:
json_normalize(datapoint, [['private_data', 'payment', 'user', 'authz_groups']])

#### private_data.payment.user.employment

In [ ]:
json_normalize(datapoint, [['private_data', 'payment', 'user', 'employment']])